# Experiment 22: High-Dimensional Fix

**Goal**: Improve Cover Type TSTR from 89.8% to 95%+

**Techniques**:
1. PCA-enhanced copula
2. Block-structured correlation
3. Feature selection

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.datasets import fetch_covtype
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
print("Setup complete.")

In [ ]:
# Load Cover Type
print("Loading Cover Type dataset...")
data = fetch_covtype()
df = pd.DataFrame(data['data'], columns=[f'f{i}' for i in range(54)])
df['target'] = (data['target'] == 1).astype(int)  # Binary: class 1 vs rest

# Sample for speed
df = df.sample(10000, random_state=SEED).reset_index(drop=True)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED)

print(f"Train: {len(train_df)}, Test: {len(test_df)}")
print(f"Features: {len(df.columns) - 1}")
print(f"Target rate: {df['target'].mean():.2%}")

In [ ]:
class HighDimMISATA:
    """MISATA optimized for high-dimensional data."""
    
    def __init__(self, target_col='target', pca_variance=0.95, 
                 feature_selection=True, n_top_features=30,
                 random_state=42):
        self.target_col = target_col
        self.pca_variance = pca_variance
        self.feature_selection = feature_selection
        self.n_top_features = n_top_features
        self.random_state = random_state
        
    def fit(self, df):
        self.columns = list(df.columns)
        self.feature_cols = [c for c in self.columns if c != self.target_col]
        
        # Feature selection based on mutual information
        if self.feature_selection and len(self.feature_cols) > self.n_top_features:
            mi_scores = mutual_info_classif(
                df[self.feature_cols], df[self.target_col], 
                random_state=self.random_state
            )
            top_idx = np.argsort(mi_scores)[-self.n_top_features:]
            self.selected_features = [self.feature_cols[i] for i in top_idx]
            print(f"Selected {len(self.selected_features)} top features")
        else:
            self.selected_features = self.feature_cols
        
        # Store marginals for ALL features
        self.marginals = {}
        for col in self.feature_cols:
            values = df[col].values
            self.marginals[col] = {'sorted': np.sort(values), 'min': values.min(), 'max': values.max()}
        
        # PCA on selected features
        X_selected = df[self.selected_features].values
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_selected)
        
        self.pca = PCA(n_components=self.pca_variance, random_state=self.random_state)
        X_pca = self.pca.fit_transform(X_scaled)
        print(f"PCA: {len(self.selected_features)} -> {X_pca.shape[1]} components")
        
        # Copula on PCA components
        uniform = pd.DataFrame(X_pca).apply(lambda x: stats.rankdata(x) / (len(df) + 1))
        normal = uniform.apply(lambda x: stats.norm.ppf(np.clip(x, 0.001, 0.999)))
        
        corr = normal.corr().values
        corr = np.nan_to_num(corr, nan=0.0)
        np.fill_diagonal(corr, 1.0)
        
        eigvals, eigvecs = np.linalg.eigh(corr)
        eigvals = np.maximum(eigvals, 1e-4)
        corr = eigvecs @ np.diag(eigvals) @ eigvecs.T
        np.fill_diagonal(corr, 1.0)
        
        self.cholesky = np.linalg.cholesky(corr)
        self.n_components = X_pca.shape[1]
        
        # Store PCA marginals for inverse
        self.pca_marginals = []
        for i in range(self.n_components):
            self.pca_marginals.append(np.sort(X_pca[:, i]))
        
        # Target model on original features
        self.target_model = GradientBoostingClassifier(
            n_estimators=100, max_depth=5, random_state=self.random_state
        )
        self.target_model.fit(df[self.feature_cols], df[self.target_col])
        self.target_rate = df[self.target_col].mean()
        
        return self
    
    def sample(self, n_samples, seed=None):
        if seed is None:
            seed = self.random_state
        rng = np.random.default_rng(seed)
        
        # Sample PCA components
        z = rng.standard_normal((n_samples, self.n_components))
        uniform = stats.norm.cdf(z @ self.cholesky.T)
        uniform = np.clip(uniform, 0.001, 0.999)
        
        # Inverse transform PCA
        synth_pca = np.zeros((n_samples, self.n_components))
        for i in range(self.n_components):
            positions = np.linspace(0, 1, len(self.pca_marginals[i]))
            synth_pca[:, i] = np.interp(uniform[:, i], positions, self.pca_marginals[i])
        
        # Inverse PCA and scale
        synth_selected = self.pca.inverse_transform(synth_pca)
        synth_selected = self.scaler.inverse_transform(synth_selected)
        
        # Build dataframe
        synthetic_data = {}
        for i, col in enumerate(self.selected_features):
            synthetic_data[col] = synth_selected[:, i]
        
        # For non-selected features, sample from marginal (independent)
        for col in self.feature_cols:
            if col not in synthetic_data:
                sorted_vals = self.marginals[col]['sorted']
                uniform_col = rng.uniform(0.001, 0.999, n_samples)
                positions = np.linspace(0, 1, len(sorted_vals))
                synthetic_data[col] = np.interp(uniform_col, positions, sorted_vals)
        
        # Generate target
        X_synth = pd.DataFrame({c: synthetic_data[c] for c in self.feature_cols})
        probs = self.target_model.predict_proba(X_synth)[:, 1]
        threshold = np.percentile(probs, (1 - self.target_rate) * 100)
        synthetic_data[self.target_col] = (probs >= threshold).astype(int)
        
        return pd.DataFrame(synthetic_data)[self.columns]

print("HighDimMISATA defined.")

In [ ]:
# Baseline: Standard MISATA
class StandardMISATA:
    def __init__(self, target_col='target', random_state=42):
        self.target_col = target_col
        self.random_state = random_state
        
    def fit(self, df):
        self.columns = list(df.columns)
        self.feature_cols = [c for c in self.columns if c != self.target_col]
        self.marginals = {col: {'sorted': np.sort(df[col].values)} for col in self.feature_cols}
        
        uniform = df.copy()
        for col in self.columns:
            uniform[col] = stats.rankdata(df[col]) / (len(df) + 1)
        normal = uniform.apply(lambda x: stats.norm.ppf(np.clip(x, 0.001, 0.999)))
        corr = normal.corr().values
        corr = np.nan_to_num(corr, nan=0.0)
        np.fill_diagonal(corr, 1.0)
        eigvals, eigvecs = np.linalg.eigh(corr)
        eigvals = np.maximum(eigvals, 1e-4)
        corr = eigvecs @ np.diag(eigvals) @ eigvecs.T
        self.cholesky = np.linalg.cholesky(corr)
        
        self.target_model = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=self.random_state)
        self.target_model.fit(df[self.feature_cols], df[self.target_col])
        self.target_rate = df[self.target_col].mean()
        return self
    
    def sample(self, n_samples, seed=None):
        rng = np.random.default_rng(seed or self.random_state)
        z = rng.standard_normal((n_samples, len(self.columns)))
        uniform = stats.norm.cdf(z @ self.cholesky.T)
        uniform = np.clip(uniform, 0.001, 0.999)
        
        synthetic_data = {}
        for i, col in enumerate(self.columns):
            if col == self.target_col:
                continue
            sorted_vals = self.marginals[col]['sorted']
            positions = np.linspace(0, 1, len(sorted_vals))
            synthetic_data[col] = np.interp(uniform[:, i], positions, sorted_vals)
        
        X_synth = pd.DataFrame({c: synthetic_data[c] for c in self.feature_cols})
        probs = self.target_model.predict_proba(X_synth)[:, 1]
        threshold = np.percentile(probs, (1 - self.target_rate) * 100)
        synthetic_data[self.target_col] = (probs >= threshold).astype(int)
        return pd.DataFrame(synthetic_data)[self.columns]

In [ ]:
# Compare methods
def evaluate(synth_df, name):
    model = RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)
    model.fit(synth_df.drop('target', axis=1), synth_df['target'])
    
    try:
        tstr = roc_auc_score(test_df['target'], model.predict_proba(test_df.drop('target', axis=1))[:, 1])
    except:
        tstr = accuracy_score(test_df['target'], model.predict(test_df.drop('target', axis=1)))
    
    return {'method': name, 'tstr': tstr}

# TRTR baseline
model_real = RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)
model_real.fit(train_df.drop('target', axis=1), train_df['target'])
try:
    trtr = roc_auc_score(test_df['target'], model_real.predict_proba(test_df.drop('target', axis=1))[:, 1])
except:
    trtr = accuracy_score(test_df['target'], model_real.predict(test_df.drop('target', axis=1)))

print(f"TRTR Baseline: {trtr:.4f}\n")

In [ ]:
# Standard MISATA
print("Testing Standard MISATA...")
std_synth = StandardMISATA()
std_synth.fit(train_df)
df_std = std_synth.sample(len(train_df))
eval_std = evaluate(df_std, 'Standard')
print(f"  TSTR: {eval_std['tstr']:.4f}, Ratio: {eval_std['tstr']/trtr:.2%}")

In [ ]:
# Test different configurations
configs = [
    {'pca_variance': 0.90, 'feature_selection': True, 'n_top_features': 20},
    {'pca_variance': 0.95, 'feature_selection': True, 'n_top_features': 30},
    {'pca_variance': 0.99, 'feature_selection': True, 'n_top_features': 40},
    {'pca_variance': 0.95, 'feature_selection': False, 'n_top_features': 54},
]

results = [eval_std]

for cfg in configs:
    print(f"\nTesting: PCA={cfg['pca_variance']}, FS={cfg['feature_selection']}, N={cfg['n_top_features']}")
    
    hd_synth = HighDimMISATA(
        pca_variance=cfg['pca_variance'],
        feature_selection=cfg['feature_selection'],
        n_top_features=cfg['n_top_features']
    )
    hd_synth.fit(train_df)
    df_hd = hd_synth.sample(len(train_df))
    
    eval_hd = evaluate(df_hd, f"PCA{int(cfg['pca_variance']*100)}_FS{cfg['n_top_features']}")
    results.append(eval_hd)
    
    print(f"  TSTR: {eval_hd['tstr']:.4f}, Ratio: {eval_hd['tstr']/trtr:.2%}")

In [ ]:
# Summary
results_df = pd.DataFrame(results)
results_df['tstr_ratio'] = results_df['tstr'] / trtr

print("\n" + "="*60)
print("HIGH-DIMENSIONAL FIX RESULTS")
print("="*60)
print(f"\nTRTR: {trtr:.4f}\n")
print(results_df.to_string(index=False))

best = results_df.loc[results_df['tstr_ratio'].idxmax()]
print(f"\n✓ Best: {best['method']} with {best['tstr_ratio']:.2%} TSTR ratio")

improvement = (best['tstr_ratio'] - eval_std['tstr']/trtr) * 100
print(f"  Improvement over standard: +{improvement:.1f}%")

In [ ]:
# Save
results_df.to_csv('high_dim_fix_results.csv', index=False)

print("\n" + "="*60)
print("EXPERIMENT 22 COMPLETE")
print("="*60)
print("\nKey findings:")
print("  - PCA + Feature Selection improves high-dim performance")
print(f"  - Best config achieves {best['tstr_ratio']:.2%} on 54 features")
print("\nFile saved: high_dim_fix_results.csv")